In [22]:
# The C2QA package is currently not published to PyPI.
# To use the package locally, add the C2QA repository's root folder to the path prior to importing c2qa.
import os
import sys
module_path = os.path.abspath(os.path.join("../.."))
if module_path not in sys.path:
    sys.path.append(module_path)

# Cheat to get MS Visual Studio Code Jupyter server to recognize Python venv
module_path = os.path.abspath(os.path.join("../../venv/Lib/site-packages"))
if module_path not in sys.path:
    sys.path.append(module_path)

In [23]:
from quspin.operators import hamiltonian  # operators
from quspin.basis import boson_basis_1d  # Hilbert space boson basis
from quspin.basis import tensor_basis, spinless_fermion_basis_1d  # Hilbert spaces
from quspin.basis import spin_basis_1d  # Hilbert space spin basis
import numpy as np
import matplotlib.pyplot as plt
from __future__ import print_function, division
from quspin.tools.Floquet import Floquet_t_vec
from quspin.tools.evolution import evolve  # ODE evolve tool
from quspin.operators import hamiltonian  # operators
from quspin.basis import boson_basis_1d  # Hilbert space boson basis
from quspin.basis import tensor_basis, spinless_fermion_basis_1d  # Hilbert spaces
from quspin.basis import spin_basis_1d  # Hilbert space spin basis
import copy
import c2qa
import util
from scipy.sparse.linalg import eigsh

### Build basis and Hamiltonian

In [24]:
Nsites = 4
Nbosons = 4
###### parameters
L_spin = Nsites
L_modes = Nsites  # system size
cutoff = Nbosons + 1  #sites+2
h = 1  # field strength
t = 1

### Build projector onto gauge conserving basis

In [25]:
P_sparse = util.gaussProjector(Nsites, Nbosons)
P_sparse

<70x10000 sparse matrix of type '<class 'numpy.complex128'>'
	with 1120 stored elements in Compressed Sparse Row format>

In [26]:
# building the two bases to tensor together
basis_spin = spin_basis_1d(L=L_spin)
basis_boson = boson_basis_1d(L=L_modes, sps=cutoff)
# print(basis_boson)
basis = tensor_basis(basis_spin, basis_boson)
# print(basis)
# print(basis.index("10","010"))

# Ratios of hopping and field

In [7]:
hopping_strength = -1
field_strength = 0
Hgaugefixed = util.build_H(hopping_strength, field_strength, L_modes, L_spin, P_sparse, basis, True)
E, psi0 = eigsh(Hgaugefixed, k=1, which='SA')
# States with weight
for i in range(len(psi0)):
    print(np.abs(psi0[i]))

psi0_notgaugefixed = P_sparse.T @ psi0

[0.35355339]
[0.35355339]
[0.35355339]
[0.35355339]
[0.35355339]
[0.35355339]
[0.35355339]
[0.35355339]


### Correlators

In [8]:
util.gauge_invariant_correlator(hopping_strength, field_strength, Nsites, psi0_notgaugefixed, Nbosons, basis)

[[ 1.76152093  2.23847907  2.23847907  1.76152093]
 [ 1.78583734  2.0244158   1.78583734 -1.        ]
 [ 1.49678554  1.49678554 -1.         -1.        ]
 [ 1.06337245 -1.         -1.         -1.        ]]


In [9]:
util.pairing_correlator(hopping_strength, field_strength, Nsites, psi0_notgaugefixed, Nbosons, basis)

## Energies

#### Hopping dominates: ground state is (1/2 |2+0> + 1/2 |0+2> + 1/sqrt(2) |1-1>)

#### Positive field dominates so |-> lowers the energy the most: ground state is |1-1>

In [8]:
elist = []
elist_freebosons = []

In [9]:
#E, psi0 = eigsh(Hgaugefixed, k=20, which='SA')
E, psi0 = np.linalg.eigh(Hgaugefixed.todense())
E_k=-2*hopping_strength*np.cos(2*np.pi*np.arange(0,L_modes,1)/L_modes)
# print(np.sort(E))
# print(np.sort(E_k))
# print(E[1]-E[0])
# print(psi0)
# psi0[:,6]

elist.append(E)
elist_freebosons.append(E_k)

In [11]:
for i in range(len(elist)):
    plt.plot(range(len(elist[i])),elist[i],"x",label="ED: eigenenergy number")
    plt.plot(range(len(elist_freebosons[i])),np.flip(elist_freebosons[i]),".",label="Analytical: momentum ($2\pi/L$)")
# plt.xlabel("ED: Eigenenergy number/ Analytical: Momentum ($2\pi/L$)")
plt.ylabel("Eigenenergy")
plt.title("ED Z2LGT no field/ Analytical free bosons. "+str(Nsites)+" sites, "+str(Nbosons)+" boson")
plt.legend()
plt.show()

# Energy gap

In [17]:
set=[]
lab = []

In [18]:
res = util.energy_gap(set, lab, -1, 1, L_modes, L_spin, P_sparse, basis, Nbosons)
set = res[0]
energy0 = res[1]
energy1 = res[2]
lab = res[3]

done 
done 
done 
done 
done 
done 
done 
done 
done 
done 
done 
done 
done 
done 
done 
done 
done 
done 
done 
done 
done 
done 
done 
done 
done 
done 
done 
done 
done 
done 
done 
done 
done 
done 
done 
done 
done 
done 
done 
done 
done 
done 
done 
done 
done 
done 
done 
done 
done 
done 


In [19]:
xlabelplot = "Value of the field term (hopping=1)"
plt.title("Z2LGT Energy gaps")
for i in range(len(set)):
    plt.plot(energy0[0],energy0[1],label="E0 "+lab[i])
for i in range(len(set)):
    plt.plot(energy1[0],energy1[1],label="E1 "+lab[i])
plt.xlabel(xlabelplot)
plt.ylabel("Eigenvalue")
plt.legend()
plt.show()

plt.title("Z2LGT Energy gap")
for i in range(len(set)):
    plt.plot(set[i][0],set[i][1],label=lab[i])
    # plt.plot(set[i][0],set[i][0]**(-1),".",label="$x^{-2}$")
plt.xlabel(xlabelplot)
plt.ylabel("Energy gap")
# plt.yscale("log")
# plt.xscale("log")
plt.legend()
plt.show()

#### field fixed varying hopping - scaling of energy gap

In [29]:
set=[]
lab = []
lab.append(str(Nsites)+" sites, "+str(Nbosons)+" bosons")

###### parameters
min = -5
max = 0
numberofvalues = 100
vals=np.linspace(min,max,numberofvalues)
vals=np.logspace(-2,2,numberofvalues)

# Different system sizes and number of bosons - choose number of bosons to be equal to the system size

gj=1
deltas=np.zeros((2,len(vals)))
energy0=np.zeros((2,len(vals)))
energy1=np.zeros((2,len(vals)))
for i in range(len(vals)):
    val = vals[i]
    hopping_strength = val
    field_strength = -1
    Hgaugefixed = util.build_H(hopping_strength, field_strength, L_modes, L_spin, P_sparse, basis, True)
    E,V = eigsh(Hgaugefixed,k=2,which='SA')
    delta=np.abs(E[1]-E[0])
    if val==0:
        print("E[0] ",E[0]," E[1] ",E[1]," delta ",delta)
    deltas[0][i]=val
    deltas[1][i]=delta

    energy0[0][i]=val
    energy0[1][i]=E[0]

    energy1[0][i]=val
    energy1[1][i]=E[1]

set.append(deltas)

In [30]:
xlabelplot = "Value of the hopping term (field=1)"
plt.title("Z2LGT Energy gaps")
for i in range(len(set)):
    plt.plot(energy0[0],energy0[1],label="E0 "+lab[i])
for i in range(len(set)):
    plt.plot(energy1[0],energy1[1],label="E1 "+lab[i])
plt.xlabel(xlabelplot)
plt.ylabel("Eigenvalue")
plt.legend()
plt.show()

plt.title("Z2LGT Energy gap")
for i in range(len(set)):
    plt.plot(np.abs(set[i][0]),np.abs(set[i][1]),label=lab[i])
    plt.plot(np.abs(set[i][0]),np.abs(set[i][0]**(2)),label="$x^{2}$")
    plt.plot(np.abs(set[i][0]),np.abs(set[i][0]**(1)),label="$x$")
plt.xlabel(xlabelplot)
plt.ylabel("Energy gap")
plt.yscale("log")
plt.xscale("log")
plt.ylim(10**(-3))

plt.legend()
plt.show()